In [ ]:
# load WebEnV
import os
import sys
import json
from train_rl import parse_args as webenv_args
from env import WebEnv  # TODO: just use webshopEnv?
import torch
from models.custom_qformer import QFormerConfigForWebshop, QFormerModelForWebshop
from PIL import Image
import torch
from train_choice_il_qformer import *
from transformers import BartForConditionalGeneration, BartTokenizer
from tqdm import tqdm
from functools import partial
import random
from minigpt4.models.minigpt4 import MiniGPT4
from minigpt4.processors.blip_processors import Blip2ImageTrainProcessor, Blip2ImageEvalProcessor
from transformers import StoppingCriteriaList
from minigpt4.conversation.conversation import StoppingCriteriaSub


def parse_args():
    parser = argparse.ArgumentParser(description="Finetune a transformers model on a text classification task")
    parser.add_argument("--model_path", type=str, default="./ckpts/web_click/epoch_9/model.pth", help="Where to store the final model.")
    parser.add_argument("--mem", type=int, default=0, help="State with memory")
    parser.add_argument("--bart_path", type=str, default='./ckpts/web_search/checkpoint-800', help="BART model path if using it")
    parser.add_argument("--bart", type=bool, default=True, help="Flag to specify whether to use bart or not (default: True)")
    parser.add_argument("--image", type=bool, default=True, help="Flag to specify whether to use image or not (default: True)")
    parser.add_argument("--softmax", type=bool, default=True, help="Flag to specify whether to use softmax sampling or not (default: True)")
    parser.add_argument("--model_name", type=str, default="qformer")
    args = parser.parse_args()

    return args

In [ ]:
FEAT_CONV = '/home/haoyang/webshop/data/feat_conv.pt'
feat_conv = torch.load(FEAT_CONV)
cache = {"asin2name": None, "name2asin": None}
DEFAULT_FILE_PATH = "../data/items_shuffle.json"
# We want to map image url to its product ASIN because in WebEnv we only have the URL
# and we need ASIN to access downloaded images
url2asin = {}
f = open(DEFAULT_FILE_PATH, 'r')
print("Line 19")
data = json.load(f)
print("Line 22")
for d in data:
    if ('product_information' not in d) or ('images' not in d) or ('asin' not in d):
        continue
    url = d['images'][0]
    if len(url) == 0 and len(d['images']) > 1:
        url = d['images'][1]
    url2asin[url] = d['asin'].upper()
data = None # save RAM
f.close()
print("Line 32")
args = webenv_args()[0]
env = WebEnv(args, split='test', feat_conv=feat_conv, cache=cache, url2asin=url2asin)
print('env loaded')
# load Model
bart_tokenizer = BartTokenizer.from_pretrained('facebook/bart-large')

In [3]:
image_processor = Blip2ImageEvalProcessor.from_config({'name': 'blip2_image_eval', 'image_size': 224})
my_data_collator = partial(data_collator, image_processor=image_processor)

In [4]:
# TODO: give an exmaple with images. one-shot example prompt
init_prompt = """Webshop
Instruction:
i would like a 3 ounce bottle of bright citrus deodorant for sensitive skin, and price lower than 50.00 dollars
[Search]

Action: search[3 ounce bright citrus deodorant sensitive skin]
Observation:
[Back to Search]
Page 1 (Total results: 50)
[Next >]
[B078GWRC1J]
Bright Citrus Deodorant by Earth Mama | Natural and Safe for Sensitive Skin, Pregnancy and Breastfeeding, Contains Organic Calendula 3-Ounce
$10.99
[B078GTKVXY]
Ginger Fresh Deodorant by Earth Mama | Natural and Safe for Sensitive Skin, Pregnancy and Breastfeeding, Contains Organic Calendula 3-Ounce
$10.99
[B08KBVJ4XN]
Barrel and Oak - Aluminum-Free Deodorant, Deodorant for Men, Essential Oil-Based Scent, 24-Hour Odor Protection, Cedar & Patchouli Blend, Gentle on Sensitive Skin (Mountain Sage, 2.7 oz, 2-Pack)
$15.95

Action: think[B078GWRC1J and B078GTKVXY are bright citrus deodorant less then 50 dollars. I can check B078GWRC1J first.]
Observation: OK.

Action: click[B078GWRC1J]
Observation:
[Back to Search]
[< Prev]
scent [assorted scents][bright citrus][calming lavender][ginger fresh][simply non-scents]
size [travel set (4-pack)][3 ounce (pack of 1)][3-ounce (2-pack)]
Bright Citrus Deodorant by Earth Mama | Natural and Safe for Sensitive Skin, Pregnancy and Breastfeeding, Contains Organic Calendula 3-Ounce
Price: $10.99
Rating: N.A.
[Description]
[Features]
[Reviews]
[Buy Now]

Action: think[For 3 ounce bottle of bright citrus deodorant for sensitive skin, the item has options 'bright citrus' and '3 ounce (pack of 1)' and seems good to buy.]
Observation: OK.

Action: click[bright citrus]
Observation: You have clicked bright citrus.

Action: click[3 ounce (pack of 1)]
Observation: You have clicked 3 ounce (pack of 1).

Action: click[Buy Now]
"""

In [5]:
def generate_prompt(exprompt, i, observation, action=None):
    if i:
      exprompt += f' {action}\nObservation: {observation}\n\nAction:'
      prompt = exprompt + f' {action}\nObservation: {observation}\n\n<Img><ImageHere></Img>\n\nAction:'
    else:
      exprompt += f'{observation}\n\nAction:'
      prompt = exprompt + f'{observation}\n\n<Img><ImageHere></Img>\n\nAction:'
    
    return prompt, exprompt

In [6]:
def bart_predict(input, model, skip_special_tokens=True, **kwargs):
    input_ids = bart_tokenizer(input)['input_ids']
    input_ids = torch.tensor(input_ids).unsqueeze(0)
    output = model.generate(input_ids, max_length=512, **kwargs)
    return bart_tokenizer.batch_decode(output.tolist(), skip_special_tokens=skip_special_tokens)


def model_generate(*args, **kwargs):
        # for 8 bit and 16 bit compatibility
        model.bfloat16()
        with model.maybe_autocast():
            output = model.llama_model.generate(*args, **kwargs)
        return output
        

def predict_v(obs, info, model, tokenizer, prompt, softmax=False, rule=False, bart_model=None):
    valid_acts = info['valid']
    if valid_acts[0].startswith('search['):
        if bart_model is None:
            return valid_acts[-1]
        else:
            goal = process_goal(obs)
            query = bart_predict(goal, bart_model, num_return_sequences=5, num_beams=5)
            # query = random.choice(query)  # in the paper, we sample from the top-5 generated results.
            query = query[0]  #... but use the top-1 generated search will lead to better results than the paper results.
            return f'search[{query}]'
            
    if rule:
        item_acts = [act for act in valid_acts if act.startswith('click[item - ')]
        if item_acts:
            return item_acts[0]
        else:
            assert 'click[buy now]' in valid_acts
            return 'click[buy now]'
                
    states = process(obs)
    actions = list(map(process, valid_acts))
    state_encodings = tokenizer(states, padding='max_length', max_length=512, truncation=True)
    action_encodings = tokenizer(actions, padding='max_length', max_length=128, truncation=True)

    raw_image = info.get('raw_image')
    batch = {
        'state_input_ids': state_encodings['input_ids'],
        'state_attention_mask': state_encodings['attention_mask'],
        'action_input_ids': action_encodings['input_ids'],
        'action_attention_mask': action_encodings['attention_mask'],
        'sizes': len(valid_acts),
        # 'images': torch.tensor(images),
        'images': info['image_feat'].tolist(),
        'raw_images': raw_image,
        'labels': 0
    }
    batch = my_data_collator([batch])
    # make batch cuda
    batch = {k: v.cuda() for k, v in batch.items()}
    state_input_ids = batch['state_input_ids']
    state_attention_mask = batch['state_attention_mask']
    action_input_ids = batch['action_input_ids']
    action_attention_mask = batch['action_attention_mask']
    sizes = batch['sizes']
    raw_images = batch['raw_images']
    images = batch['images']
    labels = batch['labels']
    # prepare image input
    image_emb, _ = model.encode_img(raw_images) 
    # prepare text input
    text = init_prompt + prompt[-(5600-len(init_prompt)):]
    # input to model
    print(len(text), image_emb.shape)
    embs = model.get_context_emb(text, [image_emb]) # batch_size, seq_len, hidden_dim
    current_max_len = embs.shape[1] + max_new_tokens
    begin_idx = max(0, current_max_len - max_length)
    embs = embs[:, begin_idx:]

    # stop words from demo
    stop_words_ids = [[835], [2277, 29937]]
    stop_words_ids = [torch.tensor(ids).to(device='cuda:{}'.format(gpu_id)) for ids in stop_words_ids]
    stopping_criteria = StoppingCriteriaList([StoppingCriteriaSub(stops=stop_words_ids)])
    print(embs.shape)
    generation_kwargs = dict(
        inputs_embeds=embs,
        max_new_tokens=max_new_tokens,
        stopping_criteria=stopping_criteria,
        num_beams=num_beams,
        do_sample=True,
        min_length=min_length,
        top_p=top_p,
        repetition_penalty=repetition_penalty,
        length_penalty=length_penalty,
        temperature=float(temperature),
    )
    output_token = model_generate(**generation_kwargs)[0]
    print("output: ", output_token)
    output_text = model.llama_tokenizer.decode(output_token, skip_special_tokens=True)
    # extract output action
    print("Output Text: {}".format(output_text))
    output_text = output_text.split('###')[0]  # remove the stop sign '###'
    output_text = output_text.split('Assistant:')[-1].strip()
    action = output_text.lstrip(' ')
    return action


def episode(model, tokenizer, idx=None, verbose=False, softmax=False, rule=False, bart_model=None):
    obs, info = env.reset(idx)
    if verbose:
        print(info['goal'])
    exprompt = ''
    action = None
    for i in range(100):
        prompt, exprompt = generate_prompt(exprompt, i, obs, action)
        if action and action.startswith('think'):
            obs = 'OK.'
        action = predict_v(obs, info, model, tokenizer, prompt, softmax=softmax, rule=rule, bart_model=bart_model)
        if verbose:
            print(action)
        obs, reward, done, info = env.step(action)
        if done:
            return reward
    return 0

In [7]:
class argsclass:
    def __init__(self):
        self.model_path = "./ckpts/web_click/epoch_9/model.pth"
        self.mem = 0
        self.bart_path='./ckpts/web_search/checkpoint-800'
        self.bart=True
        self.image=True
        self.softmax = True
        self.model_name = "qformer"

args=argsclass()

if args.mem:
    env.env.num_prev_obs = 1
    env.env.num_prev_actions = 5
    print('memory')
else:
    env.env.num_prev_obs = 0
    env.env.num_prev_actions = 0
    print('no memory')
if args.bart:
    bart_model = BartForConditionalGeneration.from_pretrained(args.bart_path)
    print('bart model loaded', args.bart_path)
else:
    bart_model = None
config = QFormerConfigForWebshop(image=True, pretrain_bert=True)
if args.model_name == "qformer":
    tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
    print(len(tokenizer))
    tokenizer.add_tokens(['[button]', '[button_]', '[clicked button]',
                            '[clicked button_]'], special_tokens=True)
    print(len(tokenizer))
    model = QFormerModelForWebshop(config, token_embed_size=len(tokenizer))
else:
    print("Model not supported")
    exit(1)

no memory
bart model loaded ./ckpts/web_search/checkpoint-800
30522
30526


Loading checkpoint shards: 100%|██████████| 2/2 [00:19<00:00,  9.88s/it]


In [8]:
print("Load minigpt4")
model_config = {
    'arch': 'minigpt4',
    'image_size': 224,
    'drop_path_rate': 0,
    'use_grad_checkpoint': False,
    'vit_precision': 'fp16',
    'freeze_vit': True,
    'freeze_qformer': True,
    'num_query_token': 32,
    'prompt': '',
    'llama_model': 'vicuna_7b',
    'model_type': 'pretrain_vicuna0',
    'max_txt_len': 160,
    'end_sym': '###',
    'low_resource': True,
    'prompt_template': '###Human: {} ###Assistant: ',
    'ckpt': 'prerained_minigpt4_7b.pth',
    'device_8bit': 0
}      
gpu_id = 0
device = torch.device("cuda:{}".format(gpu_id))
# LLM hyper-params
max_new_tokens=300
num_beams=2
min_length=1
top_p=0.9
repetition_penalty=1.05
length_penalty=1
temperature=1.0
max_length=2000
model = MiniGPT4.from_config(model_config).to('cuda:{}'.format(gpu_id))
print("Model loaded")
model.cuda()
print("Model moved to GPU")
print("choice model: {}".format(args.model_name))

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Load minigpt4


Loading checkpoint shards: 100%|██████████| 2/2 [00:17<00:00,  8.99s/it]


Loading Q-Former
Loading Q-Former Done
Load MiniGPT-4 Checkpoint: prerained_minigpt4_7b.pth
Model loaded
Model moved to GPU
choice model: qformer


In [9]:
# print('idx | reward (model), reward (rule)')
scores_softmax, scores_rule = [], []
bar = tqdm(total=500)
for i in range(500):
    print(i)
    score_softmax, score_rule = episode(model, tokenizer, idx=i, softmax=args.softmax, bart_model=bart_model), episode(model, tokenizer, idx=i, rule=True)
    # print(i, '|', score_softmax * 10, score_rule * 10)  # env score is 0-10, paper is 0-100
    scores_softmax.append(score_softmax)
    scores_rule.append(score_rule)
    bar.update(1)
bar.close()
score_softmax = sum(scores_softmax) / len(scores_softmax)
score_rule = sum(scores_rule) / len(scores_rule)
harsh_softmax = len([s for s in scores_softmax if s == 10.0])
harsh_rule = len([s for s in scores_rule if s == 10.0])
print('------')
print('avg test score (model, rule):', score_softmax * 10, score_rule * 10)
print('avg test success rate % (model, rule):', harsh_softmax / 500 * 100, harsh_rule / 500 * 100)

  0%|          | 0/500 [00:00<?, ?it/s]

0
none
5600 torch.Size([1, 32, 4096])
torch.Size([1, 1700, 4096])


/home/haoyang/anaconda3/envs/cloneweb/lib/python3.8/site-packages/transformers/generation/utils.py:1473: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(
/home/haoyang/anaconda3/envs/cloneweb/lib/python3.8/site-packages/bitsandbytes/autograd/_functions.py:318: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


RuntimeError: probability tensor contains either `inf`, `nan` or element < 0

: 

below are old codes just for reference

In [8]:
import requests
from bs4 import BeautifulSoup
from bs4.element import Comment

WEBSHOP_URL = "http://54.158.100.186:3000"
ACTION_TO_TEMPLATE = {
    'Description': 'description_page.html',
    'Features': 'features_page.html',
    'Reviews': 'review_page.html',
    'Attributes': 'attributes_page.html',
}

def clean_str(p):
  return p.encode().decode("unicode-escape").encode("latin1").decode("utf-8")


def tag_visible(element):
    ignore = {'style', 'script', 'head', 'title', 'meta', '[document]'}
    return (
        element.parent.name not in ignore and not isinstance(element, Comment)
    )


def webshop_text(session, page_type, query_string='', page_num=1, asin='', options={}, subpage='', **kwargs):
    if page_type == 'init':
      url = (
          f'{WEBSHOP_URL}/{session}'
      )
    if page_type == 'search':
      url = (
          f'{WEBSHOP_URL}/search_results/{session}/'
          f'{query_string}/{page_num}'
      )
    elif page_type == 'item':
      url = (
          f'{WEBSHOP_URL}/item_page/{session}/'
          f'{asin}/{query_string}/{page_num}/{options}'
      )
    elif page_type == 'item_sub':
      url = (
          f'{WEBSHOP_URL}/item_sub_page/{session}/'
          f'{asin}/{query_string}/{page_num}/{subpage}/{options}'
      )
    elif page_type == 'end':
      url = (
          f'{WEBSHOP_URL}/done/{session}/'
          f'{asin}/{options}'
      )
    # print(url)
    html = requests.get(url).text
    html_obj = BeautifulSoup(html, 'html.parser')
    texts = html_obj.findAll(text=True)
    visible_texts = list(filter(tag_visible, texts))
    # visible_texts = [str(text).strip().strip('\\n') for text in visible_texts]
    # if page_type == 'end': import pdb; pdb.set_trace()
    if False:
        # For `simple` mode, return just [SEP] separators
        return ' [SEP] '.join(t.strip() for t in visible_texts if t != '\n')
    else:
        # Otherwise, return an observation with tags mapped to specific, unique separators
        observation = ''
        option_type = ''
        options = {}
        asins = []
        cnt = 0
        prod_cnt = 0
        just_prod = 0
        for t in visible_texts:
            if t == '\n': continue
            if t.replace('\n', '').replace('\\n', '').replace(' ', '') == '': continue
            # if t.startswith('Instruction:') and page_type != 'init': continue
            # print(t.parent.name, t)
            if t.parent.name == 'button':  # button
                processed_t = f'\n[{t}] '
            elif t.parent.name == 'label':  # options
                if f"'{t}'" in url:
                    processed_t = f'[[{t}]]'
                    # observation = f'You have clicked {t}.\n' + observation
                else:
                    processed_t = f'[{t}]'
                options[str(t)] = option_type
                # options[option_type] = options.get(option_type, []) + [str(t)]
            elif t.parent.get('class') == ["product-link"]: # product asins
                processed_t = f'\n[{t}] '
                if prod_cnt >= 3:
                  processed_t = ''
                prod_cnt += 1
                asins.append(str(t))
                just_prod = 0
            else: # regular, unclickable text
                processed_t =  '\n' + str(t) + ' '
                if cnt < 2 and page_type != 'init': processed_t = ''
                if just_prod <= 2 and prod_cnt >= 4: processed_t = ''
                option_type = str(t)
                cnt += 1
            just_prod += 1
            observation += processed_t
        info = {}
        if options:
          info['option_types'] = options
        if asins:
          info['asins'] = asins
        if 'Your score (min 0.0, max 1.0)' in visible_texts:
          idx = visible_texts.index('Your score (min 0.0, max 1.0)')
          info['reward'] = float(visible_texts[idx + 1])
          observation = 'Your score (min 0.0, max 1.0): ' + (visible_texts[idx + 1])
        return clean_str(observation), info

class webshopEnv:
  def __init__(self):
    self.sessions = {}

  def step(self, session, action):
    done = False
    observation_ = None
    if action == 'reset':
      self.sessions[session] = {'session': session, 'page_type': 'init'}
    elif action.startswith('think['):
      observation = 'OK.'
    elif action.startswith('search['):
      assert self.sessions[session]['page_type'] == 'init'
      query = action[7:-1]
      self.sessions[session] = {'session': session, 'page_type': 'search',
                                'query_string': query, 'page_num': 1}
    elif action.startswith('click['):
      button = action[6:-1]
      if button == 'Buy Now':
        assert self.sessions[session]['page_type'] == 'item'
        self.sessions[session]['page_type'] = 'end'
        done = True
      elif button == 'Back to Search':
        assert self.sessions[session]['page_type'] in ['search', 'item_sub', 'item']
        self.sessions[session] = {'session': session, 'page_type': 'init'}
      elif button == 'Next >':
        assert False # ad hoc page limitation
        assert self.sessions[session]['page_type'] == 'search'
        self.sessions[session]['page_num'] += 1
      elif button == '< Prev':
        assert self.sessions[session]['page_type'] in ['search', 'item_sub', 'item']
        if self.sessions[session]['page_type'] == 'search':
          assert False
          self.sessions[session]['page_num'] -= 1
        elif self.sessions[session]['page_type'] == 'item_sub':
          self.sessions[session]['page_type'] = 'item'
        elif self.sessions[session]['page_type'] == 'item':
          self.sessions[session]['page_type'] = 'search'
          self.sessions[session]['options'] = {}
      elif button in ACTION_TO_TEMPLATE:
        assert self.sessions[session]['page_type'] == 'item'
        self.sessions[session]['page_type'] = 'item_sub'
        self.sessions[session]['subpage'] = button
      else:
        if self.sessions[session]['page_type'] == 'search':
          assert button in self.sessions[session].get('asins', [])  # must be asins
          self.sessions[session]['page_type'] = 'item'
          self.sessions[session]['asin'] = button
        elif self.sessions[session]['page_type'] == 'item':
          assert 'option_types' in self.sessions[session]
          assert button in self.sessions[session]['option_types'], (button, self.sessions[session]['option_types'])  # must be options
          option_type = self.sessions[session]['option_types'][button]
          if not 'options' in self.sessions[session]:
            self.sessions[session]['options'] = {}
          self.sessions[session]['options'][option_type] = button
          observation_ = f'You have clicked {button}.'
    else:
      assert False
    observation, info = webshop_text(**self.sessions[session])
    if observation_:
      observation = observation_
    self.sessions[session].update(info)
    reward = info.get('reward', 0.0)
    return observation, reward, done
env = webshopEnv()

In [9]:

from transformers import AutoModelForCausalLM, AutoTokenizer

model = AutoModelForCausalLM.from_pretrained("Writer/palmyra-small")
tokenizer = AutoTokenizer.from_pretrained("Writer/palmyra-small")

/home/ubuntu/anaconda3/envs/webshop/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
def llm3(prompt, stop=["\n"]):
    inputs = tokenizer(prompt, return_tensors='pt')
    # print("input::::::::", inputs, "finished:::::::::::::")
    tokens = model.generate(
    **inputs,
    max_new_tokens=256,
    do_sample=True,
    temperature=1.0,
    top_p=1.0,
    )
    # print("model output!!!!!!!!!!!!!!!!!!!!!!!!", tokens, "finished!!!!!!!!!!!!!!!!!!")
    return tokenizer.decode(tokens[0], skip_special_tokens=True)

# ReAct

In [11]:
import sys

def webshop_run(idx, prompt, to_print=True):
  action = 'reset'
  init_prompt = prompt
  prompt = ''
  for i in range(15):
    try:
      # interact with webshop
      res = env.step(idx, action)
      # print("*************res: ",res)
      observation = res[0]
    except AssertionError:
      observation = 'Invalid action!'

    if action.startswith('think'):
      observation = 'OK.'


    if to_print:
      # print(f'||||||||||||Action: {action}\nObservation: {observation}\n')
      sys.stdout.flush()
    if i:
      prompt += f' {action}\nObservation: {observation}\n\nAction:'
    else:
      prompt += f'{observation}\n\nAction:'
    # print("--------------------prompt: ", prompt)
    if res[2]:
      return res[1]
    # print("input to llm!!!")
    # switch to llm2 for different llms and modify output processing format accordingly
    print("============================current prompt: ", init_prompt + prompt[-(5600-len(init_prompt)):])
    action = llm3(init_prompt + prompt[-(5600-len(init_prompt)):], stop=['\n']).lstrip(' ')
    #print("======================================================llm output: ", action)
    print("============================================================================================================")

  return 0

def run_episodes(prompt, n=50):
  rs = []
  cnt = 0
  for i in range(n):
    print('-----------------')
    print(i)
    try:
      r = webshop_run(f'fixed_{i}', prompt, to_print=True)
    except AssertionError:
      r = 0
      cnt += 1
    rs.append(r)
    if (i+1) % 1 == 0:
      r, sr, fr = sum(rs) / len(rs), len([_ for _ in rs if _ == 1]) / len(rs), cnt / len(rs)
      print(i+1, r, sr, fr)
      print('-------------')
  r, sr, fr = sum(rs) / len(rs), len([_ for _ in rs if _ == 1]) / n, cnt / n
  print(r, sr, fr)
  return rs

In [13]:
res1 = run_episodes(prompt1, 1)

-----------------
0


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


============================current prompt:  Webshop
Instruction:
i would like a 3 ounce bottle of bright citrus deodorant for sensitive skin, and price lower than 50.00 dollars
[Search]

Action: search[3 ounce bright citrus deodorant sensitive skin]
Observation:
[Back to Search]
Page 1 (Total results: 50)
[Next >]
[B078GWRC1J]
Bright Citrus Deodorant by Earth Mama | Natural and Safe for Sensitive Skin, Pregnancy and Breastfeeding, Contains Organic Calendula 3-Ounce
$10.99
[B078GTKVXY]
Ginger Fresh Deodorant by Earth Mama | Natural and Safe for Sensitive Skin, Pregnancy and Breastfeeding, Contains Organic Calendula 3-Ounce
$10.99
[B08KBVJ4XN]
Barrel and Oak - Aluminum-Free Deodorant, Deodorant for Men, Essential Oil-Based Scent, 24-Hour Odor Protection, Cedar & Patchouli Blend, Gentle on Sensitive Skin (Mountain Sage, 2.7 oz, 2-Pack)
$15.95

Action: think[B078GWRC1J and B078GTKVXY are bright citrus deodorant less then 50 dollars. I can check B078GWRC1J first.]
Observation: OK.

Action:

Token indices sequence length is longer than the specified maximum sequence length for this model (1474 > 1024). Running this sequence through the model will result in indexing errors
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


============================current prompt:  Webshop
Instruction:
i would like a 3 ounce bottle of bright citrus deodorant for sensitive skin, and price lower than 50.00 dollars
[Search]

Action: search[3 ounce bright citrus deodorant sensitive skin]
Observation:
[Back to Search]
Page 1 (Total results: 50)
[Next >]
[B078GWRC1J]
Bright Citrus Deodorant by Earth Mama | Natural and Safe for Sensitive Skin, Pregnancy and Breastfeeding, Contains Organic Calendula 3-Ounce
$10.99
[B078GTKVXY]
Ginger Fresh Deodorant by Earth Mama | Natural and Safe for Sensitive Skin, Pregnancy and Breastfeeding, Contains Organic Calendula 3-Ounce
$10.99
[B08KBVJ4XN]
Barrel and Oak - Aluminum-Free Deodorant, Deodorant for Men, Essential Oil-Based Scent, 24-Hour Odor Protection, Cedar & Patchouli Blend, Gentle on Sensitive Skin (Mountain Sage, 2.7 oz, 2-Pack)
$15.95

Action: think[B078GWRC1J and B078GTKVXY are bright citrus deodorant less then 50 dollars. I can check B078GWRC1J first.]
Observation: OK.

Action:

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


============================current prompt:  Webshop
Instruction:
i would like a 3 ounce bottle of bright citrus deodorant for sensitive skin, and price lower than 50.00 dollars
[Search]

Action: search[3 ounce bright citrus deodorant sensitive skin]
Observation:
[Back to Search]
Page 1 (Total results: 50)
[Next >]
[B078GWRC1J]
Bright Citrus Deodorant by Earth Mama | Natural and Safe for Sensitive Skin, Pregnancy and Breastfeeding, Contains Organic Calendula 3-Ounce
$10.99
[B078GTKVXY]
Ginger Fresh Deodorant by Earth Mama | Natural and Safe for Sensitive Skin, Pregnancy and Breastfeeding, Contains Organic Calendula 3-Ounce
$10.99
[B08KBVJ4XN]
Barrel and Oak - Aluminum-Free Deodorant, Deodorant for Men, Essential Oil-Based Scent, 24-Hour Odor Protection, Cedar & Patchouli Blend, Gentle on Sensitive Skin (Mountain Sage, 2.7 oz, 2-Pack)
$15.95

Action: think[B078GWRC1J and B078GTKVXY are bright citrus deodorant less then 50 dollars. I can check B078GWRC1J first.]
Observation: OK.

Action:

IndexError: index out of range in self